In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 필요한 모듈 생성

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 데이터 불러오기

In [17]:
train = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/YBIGTA/교육세션/20220719/train (1).csv')
train = train.drop('id', axis=1)

In [18]:
print(train.shape)
train.head()

(17480, 15)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0


# Feature Engineering

## 결측치 확인

In [19]:
# 두 코드 모두 가능!
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             17480 non-null  int64 
 1   workclass       15644 non-null  object
 2   fnlwgt          17480 non-null  int64 
 3   education       17480 non-null  object
 4   education.num   17480 non-null  int64 
 5   marital.status  17480 non-null  object
 6   occupation      15637 non-null  object
 7   relationship    17480 non-null  object
 8   race            17480 non-null  object
 9   sex             17480 non-null  object
 10  capital.gain    17480 non-null  int64 
 11  capital.loss    17480 non-null  int64 
 12  hours.per.week  17480 non-null  int64 
 13  native.country  16897 non-null  object
 14  target          17480 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 2.0+ MB


age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
target               0
dtype: int64

## 결측치 제거

In [20]:
train_1 = train.dropna()
print("결측치 제거 전 데이터 개수 : ", train.shape[0])
print("결측치 제거 후 데이터 개수 : ", train_1.shape[0])

결측치 제거 전 데이터 개수 :  17480
결측치 제거 후 데이터 개수 :  15081


In [21]:
# 결측치가 모두 사라진것을 볼 수 있음
train_1.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
target            0
dtype: int64

## 결측치 대체

In [22]:
####결측치를 0으로 대체####
train_2 = train.fillna(0)
print("결측치 대체 전 데이터 개수 : ", train.shape[0])
print("결측치 대체 후 데이터 개수 : ", train_2.shape[0])

결측치 대체 전 데이터 개수 :  17480
결측치 대체 후 데이터 개수 :  17480


- 결측치를 0으로 대체해줌으로써 데이터 개수는 기존 데이터와 같은것을 볼 수 있음

In [23]:
####결측치를 Mean or Median으로 대체####
train_3 = train.fillna(train.mean())
train_4 = train.fillna(train.median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
####결측치를 가장 빈도수가 높은 값으로 대체####
print(train['workclass'].value_counts())
print(train['occupation'].value_counts())
print(train['native.country'].value_counts())

Private             11568
Self-emp-not-inc     1272
Local-gov            1053
State-gov             659
Self-emp-inc          594
Federal-gov           485
Never-worked            7
Without-pay             6
Name: workclass, dtype: int64
Exec-managerial      2113
Craft-repair         2101
Prof-specialty       2085
Adm-clerical         1893
Sales                1829
Other-service        1677
Machine-op-inspct    1040
Transport-moving      785
Handlers-cleaners     695
Farming-fishing       508
Tech-support          475
Protective-serv       350
Priv-house-serv        83
Armed-Forces            3
Name: occupation, dtype: int64
United-States                 15393
Mexico                          355
Philippines                     108
Germany                          75
Canada                           63
Puerto-Rico                      59
El-Salvador                      58
Cuba                             49
India                            47
England                          46
Jamaica

- 일단 결측치가 존재하는 Feature에서 어떤 unique값이 가장 빈도수가 높은지 찾아봄
    - workclass : Private
    - occupation : Exec-managerial
    - native.country : United-States

In [25]:
# 빈도수가 높은 값으로 채워줌
train_5 = train.copy()
train_5.loc[train['workclass'] != train['workclass'],'workclass'] = 'Private'
train_5.loc[train['occupation'] != train['occupation'],'occupation'] = 'Exec-managerial'
train_5.loc[train['native.country'] != train['native.country'],'native.country'] = 'United-States'
print("결측값 대체 전 workclass의 Private 값 개수 : ", train.workclass.value_counts()[0])
print("결측값 대체 후 workclass의 Private 값 개수 : ", train_5.workclass.value_counts()[0])

결측값 대체 전 workclass의 Private 값 개수 :  11568
결측값 대체 후 workclass의 Private 값 개수 :  13404


- 결측치를 빈도수가 가장 높았던 Private으로 대체해줌으로써, Private값의 개수가 늘었음을 확인할 수 있음

In [26]:
####결측치를 임의의 값으로 대체####
train_6 = train.fillna(method = 'pad')
train_7 = train.fillna(method = 'bfill')

- pad : 특정한 수치를사용하지 않고, 결측치의 바로 앞에 있는 value를 채워넣는 방식
- bfill : pad와는 반대로 각 결측치의 바로 뒤의 값으로 채워넣는 방법

# 지표변수 (Indicator variables)

In [27]:
train['age']

0        32
1        33
2        46
3        23
4        55
         ..
17475    35
17476    30
17477    71
17478    41
17479    72
Name: age, Length: 17480, dtype: int64

In [28]:
oldness = []
for a in train['age']:
    if (17 <= a <= 30):
        oldness.append('young')
    elif (31 <= a <= 50):
        oldness.append('middle')
    else:
        oldness.append('old')

train['oldness'] = oldness

In [29]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old


## Feature Split

In [30]:
train['date'] = '2022-02-03'
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness,date
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle,2022-02-03
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young,2022-02-03
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old,2022-02-03


In [31]:
year = []
month = []
day = []

for d in train['date'].str.split("-"):
    year.append(d[0])
    month.append(d[1])
    day.append(d[2])
train['year'] = year
train['month'] = month
train['day'] = day

In [32]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target,oldness,date,year,month,day
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03,2022,02,03
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1,middle,2022-02-03,2022,02,03
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0,middle,2022-02-03,2022,02,03
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0,young,2022-02-03,2022,02,03
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0,old,2022-02-03,2022,02,03


## 스케일링 Scaling

In [33]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

In [34]:
# MinMax Scaling
mm_scaler = mm_scaler.fit_transform(train['age'].values.reshape(-1,1))
train['mm_scaled_age'] = mm_scaler

# Standard Scaling
sd_scaler = sd_scaler.fit_transform(train['age'].values.reshape(-1,1))
train['sd_scaled_age'] = sd_scaler


In [35]:
train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,...,hours.per.week,native.country,target,oldness,date,year,month,day,mm_scaled_age,sd_scaled_age
0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,0,middle,2022-02-03,2022,02,03,0.205479,-0.477370
1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,40,United-States,1,middle,2022-02-03,2022,02,03,0.219178,-0.406343
2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,0,middle,2022-02-03,2022,02,03,0.397260,0.517004
3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,...,30,United-States,0,young,2022-02-03,2022,02,03,0.082192,-1.116610
4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,...,40,United-States,0,old,2022-02-03,2022,02,03,0.520548,1.156244


## 라벨 인코딩 (Label Encoding)

In [36]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [37]:
encoded = encoder.fit_transform(train['education'])

In [38]:
train['education_class'] = encoded

In [39]:
train.education_class.value_counts()

11    5566
15    3905
9     2842
12     921
8      724
1      664
7      588
0      510
5      378
14     301
6      294
2      249
10     214
4      197
3       96
13      31
Name: education_class, dtype: int64

## 원-핫 인코딩 (One-Hot Encoding)

In [40]:
pd.get_dummies(train['education']).head()

,10th,11th,12th,1st-4th,5th-6th,7th-8th,9th,Assoc-acdm,Assoc-voc,Bachelors,Doctorate,HS-grad,Masters,Preschool,Prof-school,Some-college
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


#과제

##Titanic 데이터를 가지고 5 개의 FE 방법들 직접 사용해보기 (오늘 피피티에 소개되지 않은 FE 방법도 괜찮음)

타이타닉 불러오기

In [4]:
titanic = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/YBIGTA/교육세션/20220719/titanic.csv')
titanic.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [41]:
titanic.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,8,0.015625
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,108,0.210938
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,7,0.013672
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,8,0.015625
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,22,0.042969


In [42]:
titanic.isnull().sum()

PassengerId         0
Pclass              0
Name                0
Sex                 0
Age                86
SibSp               0
Parch               0
Ticket              0
Fare                1
Cabin             327
Embarked            0
FareInt             0
mm_scaled_fare      0
dtype: int64

### 1. Imputation - 타이타닉 결측치 Median으로 대체

In [43]:
titanic_imp = titanic.fillna(titanic.median())
titanic_imp.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,7,0.013672
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,7,0.013672
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,9,0.017578
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,8,0.015625
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,12,0.023438


In [44]:
titanic_imp.isnull().sum()

PassengerId         0
Pclass              0
Name                0
Sex                 0
Age                 0
SibSp               0
Parch               0
Ticket              0
Fare                0
Cabin             327
Embarked            0
FareInt             0
mm_scaled_fare      0
dtype: int64

In [45]:
titanic_imp = titanic_imp.fillna("DEFAULT")
titanic_imp.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,DEFAULT,Q,7,0.013672
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,DEFAULT,S,7,0.013672
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,DEFAULT,Q,9,0.017578
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,DEFAULT,S,8,0.015625
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,DEFAULT,S,12,0.023438


### 2. Indicator Variables - Expensive 컬럼 추가하기

In [46]:
expensive = []
for fare in titanic['Fare']:
    if (fare > 10):
        expensive.append('expensive')
    else:
        expensive.append('cheap')

titanic_i_var = titanic
titanic_i_var['expensive'] = expensive

titanic_i_var.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare,expensive
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,7,0.013672,cheap
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,7,0.013672,cheap
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,9,0.017578,cheap
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,8,0.015625,cheap
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,12,0.023438,expensive


In [47]:
titanic_i_var.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare,expensive
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,8,0.015625,cheap
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,108,0.210938,expensive
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,7,0.013672,cheap
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,8,0.015625,cheap
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,22,0.042969,expensive


### 3. Feature Split - Ticket prefix
Ticket 이름 앞에 붙는 String이 좌석/행동과 연관이 있다는 가정


In [48]:
titanic_f_split = titanic

ticket_seller = []

for t in titanic['Ticket'].str.split(" "):
  if (len(t) > 1):
    ticket_seller.append(t[0])
  else:
    ticket_seller.append("")

titanic_f_split['ticket_seller'] = ticket_seller

titanic_f_split.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare,expensive,ticket_seller
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,7,0.013672,cheap,
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,7,0.013672,cheap,
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,9,0.017578,cheap,
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,8,0.015625,cheap,
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,12,0.023438,expensive,


In [49]:
titanic_f_split.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare,expensive,ticket_seller
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,8,0.015625,cheap,A.5.
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,108,0.210938,expensive,PC
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,7,0.013672,cheap,SOTON/O.Q.
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,8,0.015625,cheap,
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,22,0.042969,expensive,


### 4. Scaling - fare min max 스케일링

In [50]:
from sklearn.preprocessing import MinMaxScaler
import math

mm_scaler = MinMaxScaler()



In [15]:
def toInt(x):
  return int(x)

titanic_scaling = titanic
titanic_scaling['FareInt'] = titanic['Fare'].fillna(0).apply(toInt)

mm_scaler = mm_scaler.fit_transform(titanic_scaling['FareInt'].values.reshape(-1,1))
titanic_scaling['mm_scaled_fare'] = mm_scaler
titanic_scaling.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,7,0.013672
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,7,0.013672
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,9,0.017578
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,8,0.015625
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,12,0.023438


In [16]:
titanic_scaling.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareInt,mm_scaled_fare
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,8,0.015625
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,108,0.210938
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,7,0.013672
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,8,0.015625
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,22,0.042969


### 5. One-Hot Coding - Age, Fare


In [11]:
titanic_one_hot_coding = titanic
pd.get_dummies(titanic_one_hot_coding['Age']).head()

,0.17,0.33,0.75,0.83,0.92,1.00,2.00,3.00,5.00,6.00,...,58.00,59.00,60.00,60.50,61.00,62.00,63.00,64.00,67.00,76.00
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
pd.get_dummies(titanic_one_hot_coding['Fare']).tail()

,0.0000,3.1708,6.4375,6.4958,6.9500,7.0000,7.0500,7.2250,7.2292,7.2500,...,151.5500,164.8667,211.3375,211.5000,221.7792,227.5250,247.5208,262.3750,263.0000,512.3292
413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
